the following code is intended to process the road map images

In [1]:
import glob

In [2]:
# access image filenames

filenames = glob.glob('images/source_images/road_images/*')

In [3]:
print("Num_Images: ",len(filenames))

Num_Images:  1109


# I will be using code found from [this](https://towardsdatascience.com/an-end-to-end-introduction-to-gans-bf253f1fa52f) GAN introduction to test the formatting of my images.

test github access token